In [1]:
import sys
import os
from torch.utils.data import DataLoader
sys.path.insert(0,'/')
concordance = []
brier = []
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import TRiskSurv.pytorch_pretrained_bert as Bert

from TRiskSurv. pytorch_pretrained_bert import optimizer
import sklearn.metrics as skm
from torch.utils.data.dataset import Dataset
from TRiskSurv.ModelPkg.utils import *

sys.path.insert(0, '/ModelPkg/')
from TRiskSurv.ModelPkg.BEHRTsurv import *
from TRiskSurv.ModelPkg.Data_Deterministic import *
from TRiskSurv.ModelPkg.utils import *
import matplotlib as plt

from torch import optim as toptimizer

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import torch

from TRiskSurv.ModelPkg.TriSched import *

from TRiskSurv.ModelPkg.CPHloss import *


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
def trainloop(e, binaryFlag=False):
    if binaryFlag:
        print('binary on...')

    dataset_train = Binary_DeepSurvDataset(BertVocab['token2idx'], train, global_params['max_len_seq'], max_age=110, year=True, age_symbol=None, min_visit=5)


    dl_train = DataLoader(dataset_train, global_params['batch_size'], shuffle=True)


    
    model.train()
    model.binary=binaryFlag
    tr_loss = 0
    temp_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    cnt = 0

    for step, batch in enumerate(dl_train):
        cnt += 1

        
        age_ids, input_ids, posi_ids, segment_ids, attMask, time2event, label, labelfloat = batch

        age_ids = age_ids.to(global_params['device'])
        input_ids = input_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        time2event = time2event.to(global_params['device'])
        label = label.to(global_params['device'])
        labelfloat = labelfloat.to(global_params['device'])

        logits , loss= model(input_ids, age_ids, segment_ids, posi_ids, attMask, label, labelfloat, time2event)

        if global_params['gradient_accumulation_steps'] > 1:
            loss = loss / global_params['gradient_accumulation_steps']
        loss.backward()

        temp_loss += loss.item()
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        if step % 100 == 0:
            if BINFLAG:
                prec, a, b = precision(logits, labelfloat)
            else:
                prec = -1
            print("epoch: {}\t| Cnt: {}\t| Loss: {}\t| precision: {}".format(e, cnt, temp_loss / 100, prec))
            temp_loss = 0

        if (step + 1) % global_params['gradient_accumulation_steps'] == 0:
            optim.step()
        optim.zero_grad()
    del dl_train
    return e



def evaluationloop(binaryFlag=False):
    if binaryFlag:
        print('binary on...')


    dataset_test = Binary_DeepSurvDataset(BertVocab['token2idx'], test, global_params['max_len_seq'], max_age=110, year=True, age_symbol=None, min_visit=5)

    dl_test = DataLoader(dataset_test, global_params['batch_size'], shuffle=False)

    model.eval()
    y = []
    y_label = []
    loss_temp = 0
    model.binary=binaryFlag
    y_time = []
    
    for step, batch in enumerate(dl_test):
        model.eval()

        
        age_ids, input_ids, posi_ids, segment_ids, attMask, time2event, label, labelfloat = batch

        age_ids = age_ids.to(global_params['device'])
        input_ids = input_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        time2event = time2event.to(global_params['device'])
        label = label.to(global_params['device'])
        labelfloat = labelfloat.to(global_params['device'])

        with torch.no_grad():
            logits , loss= model(input_ids, age_ids, segment_ids, posi_ids, attMask, label,labelfloat, time2event)

        logits = logits.cpu()
        labelfloat = labelfloat.cpu()
        time2event = time2event.cpu()
        if step % 50 == 0:
            print(step)
        y_label.append(labelfloat)
        y.append(logits)
        y_time.append(time2event)
        loss_temp = loss_temp+loss.item()
    y_label = torch.cat(y_label, dim=0)
    y = torch.cat(y, dim=0)
    y_time = torch.cat(y_time, dim=0)
    
    if BINFLAG:
        tempprc, output, label = precision_test(y, y_label)

        tempauroc, output, label = roc_auc(y, y_label)

    else:
        tempprc, output, label = precision_test(y, y_label)

        tempauroc = cindex(y, y_label, y_time)
    
        
    
    return tempprc, tempauroc, loss_temp
    



In [4]:
def validationloop(binaryFlag=False):
    if binaryFlag:
        print('binary on...')

    dataset_test = Binary_DeepSurvDataset(BertVocab['token2idx'], valid, global_params['max_len_seq'], max_age=110, year=True, age_symbol=None, min_visit=5)

    dl_test = DataLoader(dataset_test, global_params['batch_size']*4, shuffle=False)

    model.eval()
    y = []
    y_label = []
    loss_temp = 0
    model.binary=binaryFlag
    y_time = []
    
    for step, batch in enumerate(dl_test):
        model.eval()

        
        age_ids, input_ids, posi_ids, segment_ids, attMask, time2event, label, labelfloat = batch

        age_ids = age_ids.to(global_params['device'])
        input_ids = input_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        time2event = time2event.to(global_params['device'])
        label = label.to(global_params['device'])
        labelfloat = labelfloat.to(global_params['device'])

        with torch.no_grad():
            logits , loss= model(input_ids, age_ids, segment_ids, posi_ids, attMask, label,labelfloat, time2event)

        logits = logits.cpu()
        labelfloat = labelfloat.cpu()
        time2event = time2event.cpu()
        if step % 50 == 0:
            print(step)
        y_label.append(labelfloat)
        y.append(logits)
        y_time.append(time2event)
        loss_temp = loss_temp+loss.item()
    y_label = torch.cat(y_label, dim=0)
    y = torch.exp(torch.cat(y, dim=0))
    y_time = torch.cat(y_time, dim=0)
    
    

        
    
    return y_label, y, y_time
    



In [5]:


def trainloop_bh(binaryFlag=False, traind=None):
    if traind is not None:
        train = traind
    if binaryFlag:
        print('binary on...')

    dataset_test = Binary_DeepSurvDataset(BertVocab['token2idx'], train, global_params['max_len_seq'], max_age=110, year=True, age_symbol=None, min_visit=5)


    dl_test = DataLoader(dataset_test, int(global_params['batch_size']*2.5), shuffle=False)

    model.eval()
    y = []
    y_label = []
    loss_temp = 0
    model.binary=binaryFlag
    y_time = []
    
    for step, batch in enumerate(dl_test):
        model.eval()

        
        age_ids, input_ids, posi_ids, segment_ids, attMask, time2event, label, labelfloat = batch

        age_ids = age_ids.to(global_params['device'])
        input_ids = input_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        time2event = time2event.to(global_params['device'])
        label = label.to(global_params['device'])
        labelfloat = labelfloat.to(global_params['device'])

        with torch.no_grad():
            logits , loss= model(input_ids, age_ids, segment_ids, posi_ids, attMask, label,labelfloat, time2event)

        logits = logits.cpu()
        labelfloat = labelfloat.cpu()
        time2event = time2event.cpu()
        if step % 50 == 0:
            print(step)
        y_label.append(labelfloat)
        y.append(logits)
        y_time.append(time2event)
        loss_temp = loss_temp+loss.item()
    y_label = torch.cat(y_label, dim=0)
    y = torch.exp(torch.cat(y, dim=0))
    y_time = torch.cat(y_time, dim=0)
    
    

        
    
    return y_label, y, y_time
    



In [6]:

def adam2(params, config=None):
    if config is None:
        config = {
            'lr': 3e-5,
            'warmup_proportion': 0.1
        }
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight', 'Eps','VAE']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in params if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in params if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    optim = Bert.optimization.BertAdam(optimizer_grouped_parameters,
                                       lr=config['lr'],
                                       warmup=config['warmup_proportion'])
    return optim

# Run binary prediction first...

In [19]:

global_params = {
    'batch_size': 224,
    'gradient_accumulation_steps': 1,
    'num_train_epochs': 3,
    'device': 'cuda:0',
    'output_dir': "SavedModels/",
    'save_model': True,
    'max_len_seq': 512,
    'max_age': 110,
    'age_year': False,
    'age_symbol': None,
    'fac': 0.1,
    'diseaseI': 1,
    'treatments': 2,
    

}
BINFLAG = True


trainpat= load_obj('Data/trainset')
testpat= load_obj('Data/testset')
validpat= load_obj('Data/validset')

optim_config = {
    'lr': 3e-5,
    'warmup_proportion': 0.1,


}
data = pd.read_parquet('test.parquet/')
data['time2event']= 15


In [ ]:
BertVocab = {}
token2idx = {'MASK': 4,
  'CLS': 3,
  'SEP': 2,
  'UNK': 1,
  'PAD': 0,
            'disease1':5,
             'disease2':6,
             'disease3':7,
             'disease4':8,
             'disease5':9,
             'disease6':10,
             'medication1':11,
             'medication2':12,
             'medication3':13,
             'medication4':14,
             'medication5':15,
             'medication6':16,
            }
idx2token = {}
for x in token2idx:
    idx2token[token2idx[x]]=x
BertVocab['token2idx']= token2idx
BertVocab['idx2token']= idx2token

In [20]:

ageVocab, _ = age_vocab(max_age=global_params['max_age'], year=True,
                        symbol=global_params['age_symbol'])



create_folder(global_params['output_dir'])
model_config = {
    'vocab_size': len(BertVocab['token2idx'].keys()),  # number of disease + symbols for word embedding
    'hidden_size': 150,  # word embedding and seg embedding hidden size
    'seg_vocab_size': 2,  # number of vocab for seg embedding
    'age_vocab_size': len(ageVocab.keys()),  # number of vocab for age embedding
    'max_position_embedding': global_params['max_len_seq'],  # maximum number of tokens
    'hidden_dropout_prob': 0.3,  # dropout rate
    'num_hidden_layers': 6,  # number of multi-head attention layers required
    'num_attention_heads': 6,  # number of attention heads
    'attention_probs_dropout_prob': 0.4,  # multi-head attention dropout rate
    'intermediate_size': 108,  # the size of the "intermediate" layer in the transformer encoder
    'hidden_act': 'gelu',
    'initializer_range': 0.02,  # parameter weight initializer range
    'num_treatment': global_params['treatments'],
    'device': global_params['device'],
    'concat_embeddings' : True,

    
}



In [21]:
cutiter= 0


train = data[data.patid.isin(trainpat)].reset_index(drop=True)
test = data[data.patid.isin(testpat)].reset_index(drop=True)
valid = data[data.patid.isin(validpat)].reset_index(drop=True)


print(len(data), len(list(trainpat) + list(testpat) + list(validpat)))




403775 403775


In [22]:
len(train), len(test), len(valid), len(train) +  len(test) + len(valid)

(273954, 30439, 99382, 403775)

In [23]:
model_config

{'vocab_size': 4124,
 'hidden_size': 150,
 'seg_vocab_size': 2,
 'age_vocab_size': 1322,
 'max_position_embedding': 512,
 'hidden_dropout_prob': 0.3,
 'num_hidden_layers': 6,
 'num_attention_heads': 6,
 'attention_probs_dropout_prob': 0.4,
 'intermediate_size': 108,
 'hidden_act': 'gelu',
 'initializer_range': 0.02,
 'num_treatment': 2,
 'device': 'cuda:0',
 'concat_embeddings': True}

In [24]:


import warnings

warnings.filterwarnings(action='ignore')
# optim_config = {
#     'lr': 5e-5,
#     'warmup_proportion': 0.1,
#     'weight_decay': 0.5
# }
# optim = optimizer.adam2(params=list(model.named_parameters()), config=optim_config)
# auc, time_cost = evaluation()
from sklearn.model_selection import KFold

aurocbest = []
auprcbest = []



conf = BertConfig(model_config)

model = BEHRT_DeepSurv(conf, num_labels=1)

# auc, roc, time_cost = evaluation()


512 150
turn on concat - cehrt structure - embeddings


In [25]:
pre_bert = os.path.join('/gpfs3/well/rahimi/users/gra027/JNb/general_model_newCutCPRD/SavedModels/',
                                 "MLM_CEHR_newcut1985_2020_DMProc__6msummary.bin")


pretrained_dict = torch.load(pre_bert, map_location='cpu')
modeld = model.state_dict()
for dictWeight in modeld:
    dictWeightsub = dictWeight
    print(dictWeightsub)
    #         print(dictWeightsub)
    if dictWeightsub in pretrained_dict and dictWeightsub not in ['bert.embeddings.catmap.weight', 'bert.embeddings.catmap.bias',  'bert.embeddings.posi_embeddings.weight']:
        modeld[dictWeight] = pretrained_dict[dictWeightsub]
    elif dictWeight in pretrained_dict and 'cls' in dictWeight:
        modeld[dictWeight] = pretrained_dict[dictWeight]
numPos = model_config['max_position_embedding']
if model_config['max_position_embedding'] > 250:
    numPos = 250
modeld['bert.embeddings.posi_embeddings.weight'][:numPos] = pretrained_dict[
                                                                     'bert.embeddings.posi_embeddings.weight'][
                                                                 :numPos]

model.load_state_dict(modeld)

model = model.to(global_params['device'])
optim = adam2(params=list(model.named_parameters()), config=optim_config)
model.binary = BINFLAG

scheduler = toptimizer.lr_scheduler.ExponentialLR(optim, 0.95, last_epoch=-1)
patience = 0


bert.embeddings.word_embeddings.weight
bert.embeddings.segment_embeddings.weight
bert.embeddings.age_embeddings.weight
bert.embeddings.posi_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.embeddings.catmap.weight
bert.embeddings.catmap.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encode

In [28]:
auprcbest =[]
aurocbest = []
best_pre = -1
patience = 0
bestauroc = -1
for e in range(50):
    fullCounter = trainloop(e, BINFLAG)

    prc, auroc ,loss_temp = evaluationloop(BINFLAG)
    fullCounter = fullCounter+1

    if auroc > bestauroc:
        patience=0
        # Save a trained model
        print("** ** * Saving best fine - tuned model ** ** * ")
        model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
        output_model_file = os.path.join(global_params['output_dir'],  'binarypredTrisk.bin')
        create_folder(global_params['output_dir'])
        if global_params['save_model']:
            torch.save(model_to_save.state_dict(), output_model_file)

        best_pre = prc
        bestauroc = auroc
    else:
        if patience % 2 == 0 and patience != 0:
            scheduler.step()
            print("LR: ", scheduler.get_lr())

        if patience >=3 and patience != 0:
            print('quitting... no gains...')
            break
        print(patience)
        patience = patience + 1
    print('loss: {}, prc : {}, auroc : {}'.format(loss_temp, prc, auroc))
print('best ever auroc: ', bestauroc)
print('best ever auprc: ', best_pre)

auprcbest.append(best_pre)
aurocbest.append(bestauroc)
torch.cuda.empty_cache()

binary on...
epoch: 0	| Cnt: 1	| Loss: 0.0070767349004745485	| precision: 0.25755369822528995
epoch: 0	| Cnt: 101	| Loss: 0.6346148502826691	| precision: 0.3024541357300482
epoch: 0	| Cnt: 201	| Loss: 0.5947180128097534	| precision: 0.6043179811708665
epoch: 0	| Cnt: 301	| Loss: 0.5719782993197441	| precision: 0.500210617573104
epoch: 0	| Cnt: 401	| Loss: 0.554108974635601	| precision: 0.5095252866485129
epoch: 0	| Cnt: 501	| Loss: 0.5529410451650619	| precision: 0.5348001345486142
epoch: 0	| Cnt: 601	| Loss: 0.5517038303613663	| precision: 0.5189245010372551
epoch: 0	| Cnt: 701	| Loss: 0.53964546084404	| precision: 0.5255610671743958
epoch: 0	| Cnt: 801	| Loss: 0.5442788770794869	| precision: 0.4695545580730964
epoch: 0	| Cnt: 901	| Loss: 0.5423697191476822	| precision: 0.5907113314066799
epoch: 0	| Cnt: 1001	| Loss: 0.5386657160520554	| precision: 0.5835202658728285
epoch: 0	| Cnt: 1101	| Loss: 0.5351114597916603	| precision: 0.590581286616105
epoch: 0	| Cnt: 1201	| Loss: 0.528542200

KeyboardInterrupt: 

# surv modelling

In [16]:

global_params = {
    'batch_size': 224,
    'gradient_accumulation_steps': 1,
    'num_train_epochs': 3,
    'device': 'cuda:0',
    'output_dir': "SavedModels/",
    'save_model': True,
    'max_len_seq': 512,
    'max_age': 110,
    'age_year': False,
    'age_symbol': None,
    'fac': 0.1,
    'diseaseI': 1,
    'treatments': 2,
    

}
BINFLAG = False

trainpat= load_obj('Data/trainset')
testpat= load_obj('Data/testset')
validpat= load_obj('Data/validset')


data = pd.read_parquet('test.parquet/')
data['time2event']= 15

In [18]:

ageVocab, _ = age_vocab(max_age=global_params['max_age'], year=True,
                        symbol=global_params['age_symbol'])


ageVocab, _ = age_vocab(max_age=global_params['max_age'], year=global_params['age_year'],
                        symbol=global_params['age_symbol'])

create_folder(global_params['output_dir'])
model_config = {
    'vocab_size': len(BertVocab['token2idx'].keys()),  # number of disease + symbols for word embedding
    'hidden_size': 150,  # word embedding and seg embedding hidden size
    'seg_vocab_size': 2,  # number of vocab for seg embedding
    'age_vocab_size': len(ageVocab.keys()),  # number of vocab for age embedding
    'max_position_embedding': global_params['max_len_seq'],  # maximum number of tokens
    'hidden_dropout_prob': 0.3,  # dropout rate
    'num_hidden_layers': 6,  # number of multi-head attention layers required
    'num_attention_heads': 6,  # number of attention heads
    'attention_probs_dropout_prob': 0.4,  # multi-head attention dropout rate
    'intermediate_size': 108,  # the size of the "intermediate" layer in the transformer encoder
    'hidden_act': 'gelu',
    'initializer_range': 0.02,  # parameter weight initializer range
    'num_treatment': global_params['treatments'],
    'device': global_params['device'],
    'concat_embeddings' : True,

    
}



In [19]:
optim_config = {
    'lr':8e-5, 
#     'weight_decay': 0.01, 
    'warmup_proportion': 0.1,
}

In [20]:
cutiter= 0



train = data[data.patid.isin(trainpat)].reset_index(drop=True)
test = data[data.patid.isin(testpat)].reset_index(drop=True)
valid = data[data.patid.isin(validpat)].reset_index(drop=True)



print(len(data), len(list(trainpat) + list(testpat) + list(validpat)))




403775 403775


In [23]:
len(train), len(test), len(valid), len(train) +  len(test) + len(valid)

(273954, 30439, 99382, 403775)

In [24]:
set(train.patid.values).intersection(set(test.patid.values)), set(valid.patid.values).intersection(set(test.patid.values)), set(train.patid.values).intersection(set(valid.patid.values))

# check if 0

(set(), set(), set())

In [26]:
import warnings

warnings.filterwarnings(action='ignore')
from sklearn.model_selection import KFold

aurocbest = []
auprcbest = []



conf = BertConfig(model_config)

model = BEHRT_DeepSurv(conf, num_labels=1)



turn on concat - cehrt structure - embeddings


In [27]:


def adam_surv(params, config=None):
    if config is None:
        config = {
            'lr': 3e-5,
            'warmup_proportion': 0.1
        }
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight', 'Eps','VAE']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in params if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in params if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    optim = Bert.optimization.BertAdam(optimizer_grouped_parameters,
                                       lr=config['lr'],
                                       warmup=config['warmup_proportion'])
    return optim

In [28]:
pre_bert = os.path.join('/gpfs3/well/rahimi/users/gra027/JNb/general_model_newCutCPRD/SavedModels/',
                                 "MLM_CEHR_newcut1985_2020_DMProc__6msummary.bin")
model.load_state_dict(torch.load(pre_bert, map_location='cpu'))


for param in model.parameters():
    param.requires_grad = False
# del optimizer
for n , param in model.named_parameters():
    if '5' in n:
        print(n)
        param.requires_grad = True
model.bert.pooler.dense.weight.requires_grad = True
model.bert.pooler.dense.bias.requires_grad = True
model.classifier.weight.requires_grad = True
model.classifier.bias.requires_grad = True

model = model.to(global_params['device'])
optim = adam_surv(params=list(model.named_parameters()), config=optim_config)
model.binary = BINFLAG


scheduler = toptimizer.lr_scheduler.ExponentialLR(optim, 0.95, last_epoch=-1)
patience = 0


bert.embeddings.word_embeddings.weight
bert.embeddings.segment_embeddings.weight
bert.embeddings.age_embeddings.weight
bert.embeddings.posi_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.embeddings.catmap.weight
bert.embeddings.catmap.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encode

In [32]:
auprcbest =[]
aurocbest = []
best_pre = -1
patience = 0
bestauroc = -1
for e in range(70):
    fullCounter = trainloop(e, BINFLAG)

    prc, auroc ,loss_temp = evaluationloop(BINFLAG)
    fullCounter = fullCounter+1

    if auroc > bestauroc:
        patience=0
        # Save a trained model
        print("** ** * Saving best fine - tuned model ** ** * ")
        model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
        output_model_file = os.path.join(global_params['output_dir'],  'survmodel.bin')
        create_folder(global_params['output_dir'])
        if global_params['save_model']:
            torch.save(model_to_save.state_dict(), output_model_file)

        best_pre = prc
        bestauroc = auroc
    else:


        if patience >=2 and patience <=4:
            scheduler.step()
            print("LR: ", scheduler.get_lr())
        elif patience>4:
            print("LR: ", scheduler.get_lr())

            print('quitting... no gains...')
            break
        print(patience)
        patience = patience + 1
#     scheduler.step()
    print('lr: ' , scheduler.get_lr())
    print('loss: {}, prc : {}, auroc : {}'.format(loss_temp, prc, auroc))
print('best ever auauroc: ', best_pre)
auprcbest.append(best_pre)
aurocbest.append(bestauroc)
torch.cuda.empty_cache()

epoch: 0	| Cnt: 1	| Loss: 0.04930774211883545	| precision: -1
epoch: 0	| Cnt: 101	| Loss: 4.623553133010864	| precision: -1
epoch: 0	| Cnt: 201	| Loss: 4.474506106376648	| precision: -1
epoch: 0	| Cnt: 301	| Loss: 4.4673861885070805	| precision: -1
epoch: 0	| Cnt: 401	| Loss: 4.441414451599121	| precision: -1
epoch: 0	| Cnt: 501	| Loss: 4.443978734016419	| precision: -1
epoch: 0	| Cnt: 601	| Loss: 4.430216579437256	| precision: -1
epoch: 0	| Cnt: 701	| Loss: 4.441653504371643	| precision: -1
epoch: 0	| Cnt: 801	| Loss: 4.426389131546021	| precision: -1
epoch: 0	| Cnt: 901	| Loss: 4.418193845748902	| precision: -1
epoch: 0	| Cnt: 1001	| Loss: 4.409481201171875	| precision: -1
epoch: 0	| Cnt: 1101	| Loss: 4.40348819732666	| precision: -1
epoch: 0	| Cnt: 1201	| Loss: 4.395103621482849	| precision: -1
0
50
100
** ** * Saving best fine - tuned model ** ** * 
lr:  [8e-05, 8e-05]
loss: 606.3943476676941, prc : 0.9156612339944864, auroc : 0.7961325119107817
epoch: 1	| Cnt: 1	| Loss: 0.04428609

epoch: 8	| Cnt: 901	| Loss: 4.312656083106995	| precision: -1
epoch: 8	| Cnt: 1001	| Loss: 4.33312424659729	| precision: -1
epoch: 8	| Cnt: 1101	| Loss: 4.326858277320862	| precision: -1
epoch: 8	| Cnt: 1201	| Loss: 4.332900824546814	| precision: -1
0
50
100
** ** * Saving best fine - tuned model ** ** * 
lr:  [8e-05, 8e-05]
loss: 603.9590239524841, prc : 0.9127295434270675, auroc : 0.8087958371256972
epoch: 9	| Cnt: 1	| Loss: 0.04247371673583984	| precision: -1
epoch: 9	| Cnt: 101	| Loss: 4.310381450653076	| precision: -1
epoch: 9	| Cnt: 201	| Loss: 4.312988433837891	| precision: -1
epoch: 9	| Cnt: 301	| Loss: 4.317403440475464	| precision: -1
epoch: 9	| Cnt: 401	| Loss: 4.309706215858459	| precision: -1
epoch: 9	| Cnt: 501	| Loss: 4.309112796783447	| precision: -1
epoch: 9	| Cnt: 601	| Loss: 4.303022751808166	| precision: -1
epoch: 9	| Cnt: 701	| Loss: 4.319072790145874	| precision: -1
epoch: 9	| Cnt: 801	| Loss: 4.330215754508973	| precision: -1
epoch: 9	| Cnt: 901	| Loss: 4.3125695

epoch: 17	| Cnt: 301	| Loss: 4.268366560935974	| precision: -1
epoch: 17	| Cnt: 401	| Loss: 4.272680749893189	| precision: -1
epoch: 17	| Cnt: 501	| Loss: 4.271491303443908	| precision: -1
epoch: 17	| Cnt: 601	| Loss: 4.278654737472534	| precision: -1
epoch: 17	| Cnt: 701	| Loss: 4.273997535705567	| precision: -1
epoch: 17	| Cnt: 801	| Loss: 4.286304588317871	| precision: -1
epoch: 17	| Cnt: 901	| Loss: 4.275985498428344	| precision: -1
epoch: 17	| Cnt: 1001	| Loss: 4.282987775802613	| precision: -1
epoch: 17	| Cnt: 1101	| Loss: 4.268408346176147	| precision: -1
epoch: 17	| Cnt: 1201	| Loss: 4.271391277313232	| precision: -1
0
50
100
LR:  [6.51605e-05, 6.51605e-05]
quitting... no gains...
best ever auauroc:  0.9167787103076941


# surv validation

In [131]:


import warnings

warnings.filterwarnings(action='ignore')

from sklearn.model_selection import KFold

aurocbest = []
auprcbest = []



conf = BertConfig(model_config)

model = BEHRTSurvModel(conf, num_labels=1)



In [133]:

fullBert = os.path.join(global_params['output_dir'],  'survmodel.bin')

pretrained_dict = torch.load(fullBert, map_location='cpu')
net_dict = model.state_dict()
print(len(net_dict))
# # 1. filter out unnecessary keys
pretrained_dict2 = {k: v for k, v in pretrained_dict.items() if k in net_dict }
# # 2. overwrite entries in the existing state dict
print(pretrained_dict2.keys())

print(len(pretrained_dict2))
net_dict.update(pretrained_dict2) 

model.load_state_dict(net_dict)


model = model.to(global_params['device'])
optim = adam_surv(params=list(model.named_parameters()), config=optim_config)
model.binary = BINFLAG

scheduler = TriStageLRScheduler(optim,  5,5,50, 0.00001,0.000005,-1 )
patience = 0


106
dict_keys(['bert.embeddings.word_embeddings.weight', 'bert.embeddings.segment_embeddings.weight', 'bert.embeddings.age_embeddings.weight', 'bert.embeddings.posi_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.output.dense.weight', 'bert.encoder.layer.0.output.dense.bias', 'bert.encoder.layer.0.output.LayerNorm.

In [134]:
auprcbest =[]
aurocbest = []
best_pre = -1
patience = 0
bestauroc = -1


e,ph, t = validationloop(False)


0


In [135]:
from lifelines import utils
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index

def get_probability(data, ph, t0, bch):
    predictions_at_t0 = np.clip(1 - predict_survival_function(data, ph, times=[t0], bch=bch).T.squeeze(), 1e-10, 1 - 1e-10)
    return predictions_at_t0
    
def predict_survival_function(test_data, ph, times, bch):
    return np.exp(-predict_cumulative_hazard(test_data, times, ph, bch))

def predict_cumulative_hazard(X, times, ph, baseline_cumulative_hazard):
    n = X.shape[0]
    times = np.atleast_1d(times).astype(float)
    v = pd.Series(ph.reshape(-1))
    col = utils._get_index(v)
    times_ = times
    times_to_evaluate_at = np.tile(times_, (n, 1))

    c_0 = utils.interpolate_at_times(baseline_cumulative_hazard, times_to_evaluate_at).T

    cumulative_hazard_ = pd.DataFrame(c_0 * v.values, columns=col, index=times_)
    return cumulative_hazard_

def get_baseline_cumulative_hazard(train):
    time_col = 'time2event'
    label_col = 'label'
    formular = "age+gen_ethnicity+imd2015_5+systolic+systolic_std+BMI+hdl+smoke+chd_history+rheumatoid_arthritis+atrial_fibrillation+ckd+migraine+lupus_erythematosus+mental_ill+hiv_aids+ED+antihtn+antipsychotic+corticosteroid"

    cph = CoxPHFitter()
    cph.fit(train, duration_col=time_col, event_col=label_col, formula=formular)
    return cph.baseline_cumulative_hazard_

In [136]:
datatabular = pd.read_parquet('/well/rahimi/users/sev854/project/BEHRT-Surv/data/QRISK-final.parquet/').rename(columns = {'event': 'label', 'time':'time2event' })
datatabular = datatabular[datatabular.diabetes==1]
train_tabular = datatabular[datatabular.patid.isin(trainpat)]
train_tabular= train_tabular.reset_index(drop=True)

get c statistics

In [137]:
t = t.view(-1).numpy()
e = e.view(-1).numpy()
ph = ph.view(-1).numpy()
bch = get_baseline_cumulative_hazard(train_tabular)
p = get_probability(data=valid, ph=ph, t0=120, bch=bch)

print('c- statistics: ', concordance_index(t, -ph, e))

np.savez('temp_results.npz', time=t, event=e,  pred = p.values, partial_hazard=ph)

# np.savez(os.path.join(save_path, 'BEHRTSurv_scratchimplement_SurvCPHtraining__postBCE_6layers.npz'), time=t, event=e, pred=p.values, partial_hazard=ph)


c- statistics:  0.7887768137541917


# baseline haz computation and survival probability estimation

In [18]:
dd = np.load('temp_results.npz')
t = dd['time']
e = dd['event']
ph = dd['partial_hazard']

In [19]:
import warnings

warnings.filterwarnings(action='ignore')

from sklearn.model_selection import KFold

aurocbest = []
auprcbest = []



conf = BertConfig(model_config)

model = BEHRTSurvModel(conf, num_labels=1)

# auc, roc, time_cost = evaluation()


In [21]:

fullBert = os.path.join(global_params['output_dir'],  'survmodel.bin')

pretrained_dict = torch.load(fullBert, map_location='cpu')
net_dict = model.state_dict()
print(len(net_dict))
# # 1. filter out unnecessary keys
pretrained_dict2 = {k: v for k, v in pretrained_dict.items() if k in net_dict }
# # 2. overwrite entries in the existing state dict
print(pretrained_dict2.keys())

print(len(pretrained_dict2))
net_dict.update(pretrained_dict2)

model.load_state_dict(net_dict)


model = model.to(global_params['device'])
optim = adam_surv(params=list(model.named_parameters()), config=optim_config)
model.binary = BINFLAG
model.eval()
scheduler = TriStageLRScheduler(optim,  5,5,50, 0.00001,0.000005,-1 )
patience = 0


106
dict_keys(['bert.embeddings.word_embeddings.weight', 'bert.embeddings.segment_embeddings.weight', 'bert.embeddings.age_embeddings.weight', 'bert.embeddings.posi_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.output.dense.weight', 'bert.encoder.layer.0.output.dense.bias', 'bert.encoder.layer.0.output.LayerNorm.

In [ ]:
def bhcomp(df_target):
    df_target2 = df_target.groupby('time').agg({'ph': 'sum', 'event': 'sum'}).sort_index(ascending=False)    .assign(ph=lambda x: x['ph'].cumsum()).pipe(lambda x: x['event']/x['ph']).fillna(0.).iloc[::-1].loc[lambda x: x.index <= 120] .rename('baseline_hazards')
    df_target2 = df_target2.cumsum().rename('baseline cumulative hazards')
    return pd.DataFrame(df_target2)



In [24]:
auprcbest =[]
aurocbest = []
best_pre = -1
patience = 0
bestauroc = -1


e_train,ph_train, t_train = trainloop_bh(False, train.sample(frac=1).reset_index(drop=True))


0


In [29]:
trainbhdf =pd.DataFrame({'ph': ph_train.numpy().flatten(), 'event': e_train.numpy().flatten(), "time": t_train.numpy().flatten()})


trainbhdf.to_parquet('ph_train.parquet')


In [30]:
trainbhdf = pd.read_parquet('ph_train.parquet')


In [31]:
trainbhdf_bh = bhcomp(trainbhdf)

In [34]:
from lifelines import utils
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index

def get_probability(data, ph, t0, bch):
    predictions_at_t0 = np.clip(1 - predict_survival_function(data, ph, times=[t0], bch=bch).T.squeeze(), 1e-10, 1 - 1e-10)
    return predictions_at_t0
    
def predict_survival_function(test_data, ph, times, bch):
    return np.exp(-predict_cumulative_hazard(test_data, times, ph, bch))

def predict_cumulative_hazard(X, times, ph, baseline_cumulative_hazard):
    n = X.shape[0]
    times = np.atleast_1d(times).astype(float)
    v = pd.Series(ph.reshape(-1))
    col = utils._get_index(v)
    times_ = times
    times_to_evaluate_at = np.tile(times_, (n, 1))

    c_0 = utils.interpolate_at_times(baseline_cumulative_hazard, times_to_evaluate_at).T

    cumulative_hazard_ = pd.DataFrame(c_0 * v.values, columns=col, index=times_)
    return cumulative_hazard_

def get_baseline_cumulative_hazard(train):
    time_col = 'time2event'
    label_col = 'label'
    formular = "age+gen_ethnicity+imd2015_5+systolic+systolic_std+BMI+hdl+smoke+chd_history+diabetes+rheumatoid_arthritis+atrial_fibrillation+ckd+migraine+lupus_erythematosus+mental_ill+hiv_aids+ED+antihtn+antipsychotic+corticosteroid"

    cph = CoxPHFitter()
    cph.fit(train, duration_col=time_col, event_col=label_col, formula=formular)
    return cph.baseline_cumulative_hazard_

In [36]:

p = get_probability(data=valid, ph=ph, t0=120, bch=trainbhdf_bh)

print('c- statistics: ', concordance_index(t, -ph, e))

np.savez('output_save_trisk_4analysis.npz', time=t, event=e, pred=p.values, partial_hazard=ph)


c- statistics:  0.7887768137541917
